In [1]:
import random

def hex_code_color():
    a = hex(random.randrange(0,256))
    b = hex(random.randrange(0,256))
    c = hex(random.randrange(0,256))
    a = a[2:]
    b = b[2:]
    c = c[2:]
    if len(a)<2:
        a = "0" + a
    if len(b)<2:
        b = "0" + b
    if len(c)<2:
        c = "0" + c
    z = a + b + c
    return "#" + z.upper()

In [9]:
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import DatetimeTickFormatter
import numpy as np
output_notebook()

# plotter
def SMA_plotter(code, df, x_axis_label):
    # create a new plot
    p = figure(title=code + " Simple Moving Averages", x_axis_label=x_axis_label, y_axis_label='Closing Price', width=800, x_axis_type='datetime')

    x_index = np.where(df.columns.values==x_axis_label)
    columns = np.delete(df.columns.values, x_index)
    print columns
    for item in columns:
        color = hex_code_color()
        if item == 'Close':
            p.line(df[x_axis_label], df[item], legend=item, line_color=color, line_width=3)
        else:
            p.line(df[x_axis_label], df[item], legend=item, line_color=color)
    
    p.xaxis.formatter=DatetimeTickFormatter(formats=dict(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    ))

    show(p, notebook_handle=True)

Loading BokehJS ...

In [82]:
# %load Moving_Average.py
"""
Created on Tue Feb 07 23:30:28 2017

@author: rkprajap
"""
import os, datetime, logging
import pandas as pd
import glob
import numpy as np

cur_dir = os.getcwd()
today = datetime.date.today()
FORMAT = '%(asctime)s - [%(levelname)s]: %(message)s'
log_file = 'log\\' + str(today) + '.log'
logging.basicConfig(format=FORMAT, filename=log_file, filemode='w', level=logging.DEBUG, datefmt='%d-%B-%Y %I:%M:%S %p')

data_files = glob.glob("data\\*.xlsx")
sma_list = [5, 10, 20, 50]
scored_df = pd.DataFrame(columns=['Code']).set_index(['Code']) #this df contains final scoring


# calculate SMA scores from a given SMA list
def SMA_scores(smalist, scored_df):
    sma_columns = ['SMA'+str(item) for item in smalist]
    sma_columns = ['Date','Close'] + sma_columns
    df = pd.DataFrame(columns=sma_columns)

    for f in data_files:
        code = f.replace('data\\', '').replace('.xlsx', '')
        stock_history = pd.read_excel(f, sheetname='Daily Data', header=0)
        for sma in sma_list:
            vars()['SMA'+str(sma)] = stock_history['Close'].rolling(window=sma,center=False).mean()
            stock_history['SMA'+str(sma)] = vars()['SMA'+str(sma)]    
            
        df = stock_history[sma_columns].tail(1)
        sorted_sma = np.sort(sma_list)

        score = 0
        for index, sma in enumerate(sorted_sma):
            if df['Close'].values[0] < df['SMA'+str(sma)].values[0]:
                score = score + (index + 1)*sma
        temp_df = pd.DataFrame([[code, score]], columns=['Code', 'SMA_Score']).set_index(['Code'])
        scored_df = scored_df.append(temp_df)
    return scored_df

#         df = stock_history[sma_columns].tail(np.max(sma_list))
#         print code, score, df['Close'].values[0] < df['SMA5'].values[0], df['Close'].values[0] < df['SMA10'].values[0], df['Close'].values[0] < df['SMA20'].values[0], df['Close'].values[0] < df['SMA50'].values[0]
#         SMA_plotter(code, df, 'Date')

scored_df = SMA_scores(sma_list, scored_df)
# print scored_df

In [95]:
def yearly_high_low_scores(df):
    df['52WH_Score'] = 0.0
    df['52WL_Score'] = 0.0
    for f in data_files:
        code = f.replace('data\\', '').replace('.xlsx', '')
        stock_history = pd.read_excel(f, sheetname='Daily Data', header=0).tail(365)
        highest_value = stock_history['Close'].max()
        lowest_value = stock_history['Close'].min()
        current_value = stock_history['Close'].tail(1).values[0]
        
        gap_2_highest = round(((highest_value - current_value)/current_value)*100,2)
        gap_2_lowest = round(((current_value - lowest_value)/current_value)*100, 2)
        
        df.set_value(code, '52WH_Score', gap_2_highest)
        df.set_value(code, '52WL_Score', gap_2_lowest)
    return df

result = yearly_high_low_scores(scored_df)
# for buy decision
result.sort_values(['SMA_Score','52WL_Score'])

            SMA_Score  52WH_Score  52WL_Score
Code                                         
ABB                 0       14.37       20.13
ACC                 0       14.96       20.45
ADANIPORTS          0       20.62       43.86
AMBUJACEM           0       16.47       21.66
APOLLOHOSP         25       21.26        8.91
ASHOKLEY            0       16.78       21.66
ASIANPAINT          0       21.94       21.02
AUROPHARMA         60       27.38       15.47
AXISBANK            5       30.41       23.46
BAJAJFINSV          0        1.61       56.31
BAJAJ_AUTO         25       11.03       21.36
BAJFINANCE          5      985.64       27.07
BANKBARODA          0       13.85       39.20
BEL                 0      137.12       34.21
BHARATFORG          5       32.44       26.32
BHARTIARTL          5        9.47       17.69
BHEL                0       67.84       41.86
BOSCHLTD            0       10.80       30.55
BPCL                0       57.29       24.27
BRITANNIA           5        9.49 